# insert_dws_campaign_cost_prd_d_inc

- Name: insert_dws_campaign_cost_prd_d_inc

- Description: insert data to dws_campaign_cost_prd_d_inc. Sum up cost of each product during each campaign, including total budget and it remaining budget for the previous 7 days.
- Data modeling reference: https://docs.google.com/spreadsheets/d/1FlbiZEBue2SCAUo1WOkdtX5WbEDyDrZ9B2uiZUSiRio/edit?gid=801113373#gid=801113373
- Target Table: dws_campaign_cost_prd_d_inc
- Source Table: dwd_ads_d_inc, dwd_budget_d_in, dwd_campaign_d_inc
- Created by: alvinxyzhang
- Created Date: 2025-04-28
- Version: v1.0

In [ ]:
from google.cloud import bigquery
import datetime
import pandas as pd

## Scheduled Task & Partition Table

- https://cloud.google.com/bigquery/docs/parameterized-queries?cloudshell=true#python
- https://cloud.google.com/bigquery/docs/creating-partitioned-tables#python
- https://cloud.google.com/bigquery/docs/samples/bigquery-query-results-dataframe

In [ ]:
# Construct a BigQuery client object.
client = bigquery.Client()

table_schema = [
        bigquery.SchemaField("date", "DATE"),
        bigquery.SchemaField("campaign_id", "INTEGER"),
        bigquery.SchemaField("product_id", "INTEGER"),
        bigquery.SchemaField("campaign_name", "STRING"),
        bigquery.SchemaField("campaign_type", "STRING"),
        bigquery.SchemaField("start_date", "DATE"),
        bigquery.SchemaField("end_date", "DATE"),
        bigquery.SchemaField("estimated_budget", "INTEGER"),
        bigquery.SchemaField("approved_budget", "INTEGER"),
        bigquery.SchemaField("real_spent_budget", "INTEGER"),
        bigquery.SchemaField("discount", "FLOAT"),
        bigquery.SchemaField("estimated_sales_increase", "FLOAT"),
        bigquery.SchemaField("count_product", "INTEGER"),
        bigquery.SchemaField("agency_or_specialist", "STRING"),
        bigquery.SchemaField("cost_in_store", "FLOAT"),
        bigquery.SchemaField("cost_flyer", "FLOAT"),
        bigquery.SchemaField("cost_community", "FLOAT"),
        bigquery.SchemaField("cost_loyalty", "FLOAT"),
        bigquery.SchemaField("cost_push", "FLOAT"),
        bigquery.SchemaField("ad_group", "INTEGER"),
        bigquery.SchemaField("cost_ad_total", "FLOAT"),
        bigquery.SchemaField("cost_ad_search", "FLOAT"),
        bigquery.SchemaField("cost_ad_display", "FLOAT"),
        bigquery.SchemaField("cost_ad_event", "FLOAT"),
        bigquery.SchemaField("cost_ad_retargeting", "FLOAT"),
        bigquery.SchemaField("cost_ad_email", "FLOAT"),
        bigquery.SchemaField("impressions_ad_total", "INTEGER"),
        bigquery.SchemaField("impressions_ad_search", "INTEGER"),
        bigquery.SchemaField("impressions_ad_display", "INTEGER"),
        bigquery.SchemaField("impressions_ad_event", "INTEGER"),
        bigquery.SchemaField("impressions_ad_retargeting", "INTEGER"),
        bigquery.SchemaField("impressions_ad_email", "INTEGER"),
        bigquery.SchemaField("clicks_ad_total", "INTEGER"),
        bigquery.SchemaField("clicks_ad_search", "INTEGER"),
        bigquery.SchemaField("clicks_ad_display", "INTEGER"),
        bigquery.SchemaField("clicks_ad_event", "INTEGER"),
        bigquery.SchemaField("clicks_ad_retargeting", "INTEGER"),
        bigquery.SchemaField("clicks_ad_email", "INTEGER"),
        bigquery.SchemaField("conversions_ad_total", "INTEGER"),
        bigquery.SchemaField("conversions_ad_search", "INTEGER"),
        bigquery.SchemaField("conversions_ad_display", "INTEGER"),
        bigquery.SchemaField("conversions_ad_event", "INTEGER"),
        bigquery.SchemaField("conversions_ad_retargeting", "INTEGER"),
        bigquery.SchemaField("conversions_ad_email", "INTEGER")
    ]

insert_query = f"""
    WITH campaign_tbl AS (
      -- campaign summary table
      SELECT campaign_id
          , campaign_name
          , campaign_type
          , start_date
          , end_date
          , estimated_budget
          , approved_budget
          , real_spent_budget
          , product_id
          , discount
          , estimated_sales_increase
          , COUNT(DISTINCT product_id) OVER (PARTITION BY campaign_id) count_product
      FROM `positive-karma-457703-i3.retail_dashboard.dwd_campaign_d_inc`
      WHERE (start_date <= @start_date_ and end_date >= @start_date_)
        # OR (start_date <= @end_date_ and end_date >= @end_date_)
    )
    , budget_tbl AS (
      -- budget summation of the past one week including today
      SELECT campaign_id
          , agency_or_specialist
          , product_id
          , SUM(cost_in_store) AS cost_in_store
          , SUM(cost_flyer) AS cost_flyer
          , SUM(cost_community) AS cost_community
          , SUM(cost_loyalty) AS cost_loyalty
          , SUM(cost_push) AS cost_push
      FROM (
        SELECT `date`
            , campaign_id
            , agency_or_specialist
            , product_id
            , SUM(IF(activity = "In-store Setup", spent, 0)) / AVG(product_list_len) AS cost_in_store
            , SUM(IF(activity = "Flyer Launch", spent, 0)) / AVG(product_list_len) AS cost_flyer
            , SUM(IF(activity = "Local Community Events", spent, 0)) / AVG(product_list_len) AS cost_community
            , SUM(IF(activity = "Loyalty Program", spent, 0)) / AVG(product_list_len) AS cost_loyalty
            , SUM(IF(activity = "Digital Push", spent, 0)) / AVG(product_list_len) AS cost_push
        FROM `positive-karma-457703-i3.retail_dashboard.dwd_budget_d_inc`
        WHERE `date` >= @start_date_
          AND `date` <= @end_date_
        GROUP BY `date`
              , campaign_id
              , agency_or_specialist
              , product_id
      ) as t0
      GROUP BY campaign_id
            , agency_or_specialist
            , product_id
    )
    , ad_tbl AS (
      -- advertise to sum the cost over the last 7 days
      SELECT campaign_id
          , ad_group
          , product_id
          , agency_or_specialist
          , SUM(cost) AS cost_ad_total
          , SUM(IF(ads_campaign_type = "online-search", cost, 0))  AS cost_ad_search
          , SUM(IF(ads_campaign_type = "online-displays", cost, 0))  AS cost_ad_display
          , SUM(IF(ads_campaign_type = "online-event", cost, 0))  AS cost_ad_event
          , SUM(IF(ads_campaign_type = "retargeting-ads", cost, 0))  AS cost_ad_retargeting
          , SUM(IF(ads_campaign_type = "email-ads", cost, 0))  AS cost_ad_email
          , SUM(impressions) AS impressions_ad_total
          , SUM(IF(ads_campaign_type = "online-search", impressions, 0))  AS impressions_ad_search
          , SUM(IF(ads_campaign_type = "online-displays", impressions, 0))  AS impressions_ad_display
          , SUM(IF(ads_campaign_type = "online-event", impressions, 0))  AS impressions_ad_event
          , SUM(IF(ads_campaign_type = "retargeting-ads", impressions, 0))  AS impressions_ad_retargeting
          , SUM(IF(ads_campaign_type = "email-ads", impressions, 0))  AS impressions_ad_email
          , SUM(clicks) AS clicks_ad_total
          , SUM(IF(ads_campaign_type = "online-search", clicks, 0))  AS clicks_ad_search
          , SUM(IF(ads_campaign_type = "online-displays", clicks, 0))  AS clicks_ad_display
          , SUM(IF(ads_campaign_type = "online-event", clicks, 0))  AS clicks_ad_event
          , SUM(IF(ads_campaign_type = "retargeting-ads", clicks, 0))  AS clicks_ad_retargeting
          , SUM(IF(ads_campaign_type = "email-ads", clicks, 0))  AS clicks_ad_email
          , SUM(conversions) AS conversions_ad_total
          , SUM(IF(ads_campaign_type = "online-search", conversions, 0))  AS conversions_ad_search
          , SUM(IF(ads_campaign_type = "online-displays", conversions, 0))  AS conversions_ad_display
          , SUM(IF(ads_campaign_type = "online-event", conversions, 0))  AS conversions_ad_event
          , SUM(IF(ads_campaign_type = "retargeting-ads", conversions, 0))  AS conversions_ad_retargeting
          , SUM(IF(ads_campaign_type = "email-ads", conversions, 0))  AS conversions_ad_email
      FROM `positive-karma-457703-i3.retail_dashboard.dwd_ads_d_inc`
      WHERE `date` >= @start_date_
        AND `date` <= @end_date_
      GROUP BY campaign_id
            , ad_group
            , product_id
            , agency_or_specialist
    )

    SELECT @end_date_ as `date`
        , campaign_tbl.campaign_id
        , campaign_tbl.product_id
        , campaign_name
        , campaign_type
        , start_date
        , end_date
        , estimated_budget
        , approved_budget
        , real_spent_budget
        , discount
        , estimated_sales_increase
        , count_product
        -- budget_tbl
        , budget_tbl.agency_or_specialist
        , cost_in_store
        , cost_flyer
        , cost_community
        , cost_loyalty
        , cost_push
        -- ads_tbl
        , ad_group
        , cost_ad_total
        , cost_ad_search
        , cost_ad_display
        , cost_ad_event
        , cost_ad_retargeting
        , cost_ad_email
        , impressions_ad_total
        , impressions_ad_search
        , impressions_ad_display
        , impressions_ad_event
        , impressions_ad_retargeting
        , impressions_ad_email
        , clicks_ad_total
        , clicks_ad_search
        , clicks_ad_display
        , clicks_ad_event
        , clicks_ad_retargeting
        , clicks_ad_email
        , conversions_ad_total
        , conversions_ad_search
        , conversions_ad_display
        , conversions_ad_event
        , conversions_ad_retargeting
        , conversions_ad_email
    FROM campaign_tbl
    LEFT JOIN budget_tbl
  -- including all planned product_id
      ON campaign_tbl.campaign_id = budget_tbl.campaign_id
      AND campaign_tbl.product_id = budget_tbl.product_id
    LEFT JOIN ad_tbl
      ON campaign_tbl.campaign_id = ad_tbl.campaign_id
      AND campaign_tbl.product_id = ad_tbl.product_id
      AND ad_tbl.campaign_id = budget_tbl.campaign_id
      AND ad_tbl.product_id = budget_tbl.product_id
      AND ad_tbl.agency_or_specialist = budget_tbl.agency_or_specialist
"""

# Step 1: Check if table exists
def check_table(table_id_, full_table_id_, schema_, partition_column_):
  try:
      table = client.get_table(full_table_id_)
      print(f"    Table {table_id_} already exists.")
  except:
      print(f"    Creating partitioned table {table_id_}...")

      table = bigquery.Table(full_table_id_, schema=schema_)
      table.time_partitioning = bigquery.TimePartitioning(
          type_=bigquery.TimePartitioningType.DAY,
          field=partition_col
      )

      table = client.create_table(table)
      print(f"Created partitioned table {full_table_id_}")

# Step 2: Insert Query
def insert_table(insert_query_, full_table_id_partition_, query_parameters_):
  job_config = bigquery.QueryJobConfig(
      destination = full_table_id_partition_,
      write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
      use_legacy_sql=False,
      query_parameters=query_parameters_
  )
  query_job_ = client.query(insert_query_, job_config=job_config)  # Make an API request.

  print('   Result:', query_job_.result())

  print('   Errors: ', query_job_.errors)

  return(query_job_)

# Step 3: Check Insert Rows
def check_insert_rows(partition_col_, full_table_id_, job_date_):
  query = f"""
      SELECT {partition_col_}, count(1) row_cnt
      FROM {full_table_id_}
      WHERE {partition_col_} = @date_
      GROUP BY {partition_col_}
  """
  job_config = bigquery.QueryJobConfig(
      query_parameters=[
          bigquery.ScalarQueryParameter("date_", "DATE", job_date_)
      ]
  )
  query_job = client.query_and_wait(query, job_config=job_config)

  row_cnt_df = query_job.to_dataframe()
  return(row_cnt_df)

# Main excution
dataset_id = 'retail_dashboard'
target_table = 'dws_campaign_cost_prd_d_inc'
partition_col = 'date'
job_date = datetime.date(2024, 1, 1)
date_interval = 6

# Full table IDs
partition = job_date.strftime("%Y%m%d")
full_table_id = f"{client.project}.{dataset_id}.{target_table}"
full_table_id_partition = f"{client.project}.{dataset_id}.{target_table}${partition}"
query_parameters = [
        bigquery.ScalarQueryParameter("start_date_", "DATE", job_date), #  - datetime.timedelta(days=date_interval)
        bigquery.ScalarQueryParameter("end_date_", "DATE", job_date)
    ]

print('- Execution: ', job_date, ' Partition: ', partition)
check_table(table_id_=target_table, full_table_id_=full_table_id, schema_ = table_schema, partition_column_ = partition_col)
query_job = insert_table(insert_query_ = insert_query, full_table_id_partition_ = full_table_id_partition, query_parameters_ = query_parameters)
print('- Finish writing to Partition ', partition)
row_cnt_df = check_insert_rows(partition_col_ = partition_col, full_table_id_ = full_table_id, job_date_ = job_date)
print('- Check: insert', str(row_cnt_df['row_cnt'].values[0]), 'rows.')

- Execution:  2024-01-01  Partition:  20240101
    Table dws_campaign_cost_prd_d_inc already exists.
   Result: <google.cloud.bigquery.table.RowIterator object at 0x7f1e6e84f5e0>
   Errors:  None
- Finish writing to Partition  20240101


IndexError: index 0 is out of bounds for axis 0 with size 0

## Iterate over Date


In [11]:
start_date = datetime.date(2024, 1, 1)
end_date = datetime.date(2025, 4, 25)
dataset_id = 'retail_dashboard'
target_table = 'dws_campaign_cost_prd_d_inc'
partition_col = 'date'
date_interval = 6
full_table_id = f"{client.project}.{dataset_id}.{target_table}"

total_row_cnt_df = pd.DataFrame()

# iterate over range of dates
while (start_date <= end_date):
  job_date = start_date

  # Full table IDs
  partition = job_date.strftime("%Y%m%d")
  full_table_id_partition = f"{client.project}.{dataset_id}.{target_table}${partition}"
  query_parameters = [
          bigquery.ScalarQueryParameter("start_date_", "DATE", job_date), #  - datetime.timedelta(days=date_interval)
          bigquery.ScalarQueryParameter("end_date_", "DATE", job_date)
      ]

  print('- Execution: ', job_date, ' Partition: ', partition)
  check_table(table_id_=target_table, full_table_id_=full_table_id, schema_ = table_schema, partition_column_ = partition_col)
  query_job = insert_table(insert_query_ = insert_query, full_table_id_partition_ = full_table_id_partition, query_parameters_ = query_parameters)
  print('- Finish writing to Partition ', partition)
  row_cnt_df = check_insert_rows(partition_col_ = partition_col, full_table_id_ = full_table_id, job_date_ = job_date)
  if row_cnt_df.shape[0] == 0:
    print('- Check: insert 0 rows.')
  else:
    print('- Check: insert', str(row_cnt_df['row_cnt'].values[0]), 'rows.')

  total_row_cnt_df = pd.concat([total_row_cnt_df, row_cnt_df], axis=0, join='outer', ignore_index=True)

  start_date += datetime.timedelta(days=1)


- Execution:  2024-01-01  Partition:  20240101
    Table dws_campaign_cost_prd_d_inc already exists.
   Result: <google.cloud.bigquery.table.RowIterator object at 0x7f1e6d407a60>
   Errors:  None
- Finish writing to Partition  20240101
- Check: insert 0 rows.
- Execution:  2024-01-02  Partition:  20240102
    Table dws_campaign_cost_prd_d_inc already exists.
   Result: <google.cloud.bigquery.table.RowIterator object at 0x7f1e6d407b80>
   Errors:  None
- Finish writing to Partition  20240102
- Check: insert 0 rows.
- Execution:  2024-01-03  Partition:  20240103
    Table dws_campaign_cost_prd_d_inc already exists.
   Result: <google.cloud.bigquery.table.RowIterator object at 0x7f1e6d404f70>
   Errors:  None
- Finish writing to Partition  20240103
- Check: insert 0 rows.
- Execution:  2024-01-04  Partition:  20240104
    Table dws_campaign_cost_prd_d_inc already exists.
   Result: <google.cloud.bigquery.table.RowIterator object at 0x7f1e6d404280>
   Errors:  None
- Finish writing to Part

<ipython-input-11-551945fa7199>:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_row_cnt_df = pd.concat([total_row_cnt_df, row_cnt_df], axis=0, join='outer', ignore_index=True)


   Result: <google.cloud.bigquery.table.RowIterator object at 0x7f1e6d4063e0>
   Errors:  None
- Finish writing to Partition  20240114
- Check: insert 157 rows.
- Execution:  2024-01-15  Partition:  20240115
    Table dws_campaign_cost_prd_d_inc already exists.
   Result: <google.cloud.bigquery.table.RowIterator object at 0x7f1e6d4049d0>
   Errors:  None
- Finish writing to Partition  20240115
- Check: insert 178 rows.
- Execution:  2024-01-16  Partition:  20240116
    Table dws_campaign_cost_prd_d_inc already exists.
   Result: <google.cloud.bigquery.table.RowIterator object at 0x7f1e6d4046d0>
   Errors:  None
- Finish writing to Partition  20240116
- Check: insert 175 rows.
- Execution:  2024-01-17  Partition:  20240117
    Table dws_campaign_cost_prd_d_inc already exists.
   Result: <google.cloud.bigquery.table.RowIterator object at 0x7f1e6f351f90>
   Errors:  None
- Finish writing to Partition  20240117
- Check: insert 165 rows.
- Execution:  2024-01-18  Partition:  20240118
    Ta

<ipython-input-11-551945fa7199>:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_row_cnt_df = pd.concat([total_row_cnt_df, row_cnt_df], axis=0, join='outer', ignore_index=True)


   Result: <google.cloud.bigquery.table.RowIterator object at 0x7f1e6f1bd4b0>
   Errors:  None
- Finish writing to Partition  20240122
- Check: insert 0 rows.
- Execution:  2024-01-23  Partition:  20240123
    Table dws_campaign_cost_prd_d_inc already exists.


...

<ipython-input-11-551945fa7199>:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_row_cnt_df = pd.concat([total_row_cnt_df, row_cnt_df], axis=0, join='outer', ignore_index=True)


   Result: <google.cloud.bigquery.table.RowIterator object at 0x7f1e6e84c0d0>
   Errors:  None
- Finish writing to Partition  20250424
- Check: insert 0 rows.
- Execution:  2025-04-25  Partition:  20250425
    Table dws_campaign_cost_prd_d_inc already exists.


<ipython-input-11-551945fa7199>:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_row_cnt_df = pd.concat([total_row_cnt_df, row_cnt_df], axis=0, join='outer', ignore_index=True)


   Result: <google.cloud.bigquery.table.RowIterator object at 0x7f1e6e84d4b0>
   Errors:  None
- Finish writing to Partition  20250425
- Check: insert 0 rows.


<ipython-input-11-551945fa7199>:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_row_cnt_df = pd.concat([total_row_cnt_df, row_cnt_df], axis=0, join='outer', ignore_index=True)


In [12]:
total_row_cnt_df

,date,row_cnt
0,2024-01-13,146
1,2024-01-14,157
2,2024-01-15,178
3,2024-01-16,175
4,2024-01-17,165
...,...,...
273,2025-04-14,118
274,2025-04-15,120
275,2025-04-16,124
276,2025-04-17,121
